# 文本分类实例

## step1 导入相关包

In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

/home/fhj/.conda/envs/ideal/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## step2 加载数据
import pandas as pd
data = pd.read_csv('./ChnSentiCorp_htl_all.csv')  # 假设数据存储在data.csv文件中
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
data = data.dropna()  # 删除缺失值
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## step3 创建Dataset

In [4]:
## 创建Dataset
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self):
        self.data = pd.read_csv('./ChnSentiCorp_htl_all.csv')  # 假设数据存储在data.csv文件中
        self.data = self.data.dropna()  # 删除缺失值

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data.iloc[idx]['review'], self.data.iloc[idx]['label']

In [5]:
dataset = MyDataset()
for i in range(3):
    print(dataset[i])  # 查看前3个样本

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)


## step4 划分数据集

In [6]:
from torch.utils.data import random_split
trainset , validset = random_split(dataset,lengths = [0.9,0.1])
len(trainset),len(validset)

(6989, 776)

## step5 创建Dataloader

In [7]:
import torch
tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')
def collate_func(batch):
    texts, labels = [],[]
    for text, label in batch:
        texts.append(text)
        labels.append(label)
    inputs= tokenizer(texts, max_length=128,padding=True, truncation=True, return_tensors='pt')
    inputs['labels'] =  torch.tensor(labels)
    return inputs

In [8]:
## 划分batch
from torch.utils.data import DataLoader  
trainloader = DataLoader(trainset,batch_size=32,shuffle=True,collate_fn=collate_func)
validloader = DataLoader(validset,batch_size=64,shuffle=False,collate_fn=collate_func)
for batch in trainloader:
    print(batch)
    break

{'input_ids': tensor([[ 101,  679, 4761,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ..., 3300,  976,  102],
        [ 101,  679, 1962,  ..., 6589,  100,  102],
        ...,
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101, 6820, 5050,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1])}


## 创建模型及优化器

In [13]:
model = AutoModelForSequenceClassification.from_pretrained('hfl/rbt3', num_labels=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
if torch.cuda.is_available():
    model.cuda() ## 将模型移到GPU上

ImportError: cannot import name 'is_torchcodec_available' from 'transformers.utils' (/home/fhj/.conda/envs/ideal/lib/python3.9/site-packages/transformers/utils/__init__.py)

## step7 训练与验证

In [12]:
import transformers
print(transformers.__version__)

4.52.4


In [ ]:
def evaluate():  ## 定义评估函数  比如准确率、精确率、召回率等
    model.eval()  # 设置模型为评估模式
    total, correct = 0, 0
    with torch.no_grad():  # 评估时不计算梯度
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()} ## 将batch数据移到GPU上
            outputs = model(**batch) ## 前向计算
            logits = outputs.logits  ## 获取模型输出的logits
            predictions = torch.argmax(logits, dim=-1) ## 获取预测结果
            total += batch['labels'].size(0)
            correct += (predictions == batch['labels']).sum().item()
    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy:.4f}")  # 打印准确率

def train(epochs=3,log_step=100): #训练3轮，每100步打印一次loss
    global_step = 0
    model.train() # 设置模型为训练模式
    for epoch in range(epochs):
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()} ## 将batch数据移到GPU上
            outputs = model(**batch) ## 前向计算
            loss = outputs.loss
            loss.backward() ## 反向传播计算梯度
            optimizer.step() ## 更新参数
            optimizer.zero_grad()  ## 清空梯度
            if global_step % log_step == 0: 
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
            global_step += 1